https://ipython.readthedocs.io/en/stable/interactive/magics.html

In [ ]:
%load_ext google.cloud.bigquery

In [ ]:
%%bigquery df_example
SELECT * FROM (
  SELECT
    [414016,
    5767168,
    89019964] AS phone_numbers
  UNION ALL
  SELECT
    [8708227,
    37637318] AS phone_numbers
  UNION ALL
  SELECT
    [9349460,
    84826376,
    9215080,
    26331650] AS phone_numbers)

In [ ]:
df_example.to_dict()

{'phone_numbers': {0: [414016, 5767168, 89019964],
  1: [8708227, 37637318],
  2: [9349460, 84826376, 9215080, 26331650]}}

In [ ]:
phone_number_strings_list = []
for phone_numbers in df_example['phone_numbers']:
  phone_number_strings = []
  for phone_number in phone_numbers:
    phone_number_string = f'{phone_number:08d}'
    phone_number_strings.append(f'(+81) {phone_number_string[:4]}-{phone_number_string[-4:]}')
  phone_number_strings_list.append(phone_number_strings)
df_example['phone_number_strings'] = phone_number_strings_list
del df_example['phone_numbers']

In [ ]:
df_example.to_dict()

{'phone_number_strings': {0: ['(+81) 0041-4016',
   '(+81) 0576-7168',
   '(+81) 8901-9964'],
  1: ['(+81) 0870-8227', '(+81) 3763-7318'],
  2: ['(+81) 0934-9460',
   '(+81) 8482-6376',
   '(+81) 0921-5080',
   '(+81) 2633-1650']}}

In [ ]:
!sudo -HE pip install pandas_gbq

     |████████████████████████████████| 143kB 37.3MB/s eta 0:00:01
     |████████████████████████████████| 24.4MB 43.3MB/s eta 0:00:01


https://console.cloud.google.com/bigquery?project=sandbox-sheng

In [ ]:
df_example.to_gbq(project_id = 'sandbox-sheng', destination_table = 'bigquery_learning.customers', if_exists = 'replace')

/home/jupyter-sheng_wei/.local/lib/python3.6/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
1it [00:04,  4.10s/it]


## IPython Magics for BigQuery
* https://cloud.google.com/bigquery/docs/visualize-jupyter
* https://googleapis.dev/python/bigquery/latest/magics.html

In [ ]:
from datetime import datetime, timezone, timedelta
# from 30 days ago to 29 days ago
start_date = datetime.now(timezone(timedelta(hours=+9))) - timedelta(days=30)
end_date = datetime.now(timezone(timedelta(hours=+9))) - timedelta(days=29)
params = {
    'start_date': start_date.strftime('%Y-%m-%d %H:%M:%S Asia/Tokyo'),
    'end_date': end_date.strftime('%Y-%m-%d %H:%M:%S Asia/Tokyo')
}

In [ ]:
print(params['start_date'])
print(params['end_date'])

2021-10-14 09:21:28 Asia/Tokyo
2021-10-15 09:21:28 Asia/Tokyo


In [ ]:
%%time
%%bigquery df --param $params
SELECT
  *
FROM
  `sandbox-sheng.fluentd.fluentd_test`
WHERE
  time >= @start_date
  AND time < @end_date
ORDER BY
  time ASC

In [ ]:
df

## Read [BigQuery Reference](https://cloud.google.com/bigquery/docs/reference/standard-sql/lexical)

* https://cloud.google.com/bigquery/docs/reference/standard-sql/arrays
* https://cloud.google.com/bigquery/docs/reference/standard-sql/string_functions

In [ ]:
%%bigquery
SELECT
  ARRAY(
  SELECT
    CONCAT('(+81) ', SUBSTR(FORMAT('%08d', phone_number), 0, 4), '-', SUBSTR(FORMAT('%08d', phone_number), -4))
  FROM
    UNNEST(phone_numbers) AS phone_number )[OFFSET(0)] AS phone_number
FROM (
  SELECT
    [414016,
    5767168,
    89019964] AS phone_numbers
  UNION ALL
  SELECT
    [8708227,
    37637318] AS phone_numbers
  UNION ALL
  SELECT
    [9349460,
    84826376,
    9215080,
    26331650] AS phone_numbers)

,phone_number
0,(+81) 0041-4016
1,(+81) 0870-8227
2,(+81) 0934-9460


https://cloud.google.com/bigquery/docs/reference/standard-sql/aggregate_functions#array_agg

In [ ]:
%%bigquery
WITH fruits AS
  (SELECT "apple" AS fruit
   UNION ALL SELECT "pear" AS fruit
   UNION ALL SELECT "banana" AS fruit)
SELECT ARRAY_AGG(fruit ORDER BY fruit) AS fruit_basket
FROM fruits

,fruit_basket
0,"[apple, banana, pear]"


In [ ]:
%%bigquery
SELECT
  name,
  ARRAY_AGG( DISTINCT phone_number
  ORDER BY
    phone_number ASC )[OFFSET(0)] AS phone_number
FROM (
  SELECT
    'Andrew' AS name,
    414016 AS phone_number
  UNION ALL
  SELECT
    'Andrew' AS name,
    5767168 AS phone_number
  UNION ALL
  SELECT
    'Andrew' AS name,
    89019964 AS phone_number
  UNION ALL
  SELECT
    'Andrew' AS name,
    8708227 AS phone_number
  UNION ALL
  SELECT
    'Andrew' AS name,
    37637318 AS phone_number
  UNION ALL
  SELECT
    'Kelly' AS name,
    9349460 AS phone_number
  UNION ALL
  SELECT
    'Kelly' AS name,
    84826376 AS phone_number
  UNION ALL
  SELECT
    'Kelly' AS name,
    9215080 AS phone_number
  UNION ALL
  SELECT
    'Kelly' AS name,
    26331650 AS phone_number)
GROUP BY
  name

,name,phone_number
0,Andrew,414016
1,Kelly,9215080
